In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

import os
import numpy as np
import cv2
import progressbar

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 40, 50

train_data_dir = '/Users/alejandro/Dropbox (Work)/Hand_Data/train/'
nb_train_samples = np.sum((len(os.listdir(train_data_dir + 'hand')), 
                           len(os.listdir(train_data_dir + 'no_hand'))))
epochs = 5
batch_size = 100
n_channels = 1
rbg = 0

In [3]:
def get_data(path, color=0):
    hands = os.listdir(path + 'hand/')
    no_hands = os.listdir(path + 'no_hand/')
    
    data = np.zeros((1, img_width, img_height, n_channels))
    
    with progressbar.ProgressBar(max_value=len(hands)) as bar:
        for i in range(len(hands)):
            img1 = cv2.imread(path + 'hand/' + hands[i], color)
            img1 = img1.reshape((1, img_width, img_height, n_channels)) // 255
            img2 = cv2.imread(path + 'no_hand/' + no_hands[i], color)
            img2 = img2.reshape((1, img_width, img_height, n_channels)) // 255
            data = np.concatenate((data, img1, img2))
            bar.update(i)
    labels = np.repeat([0, 1], len(hands))
    
    return data[1:], labels

In [4]:
path = '/Users/alejandro/Dropbox (Work)/Hand_Data/train/'
train_feat, train_lab = get_data(path, color=rbg)

100% (4527 of 4527) |#####################| Elapsed Time: 0:03:40 Time: 0:03:40


In [5]:
path = '/Users/alejandro/Dropbox (Work)/Hand_Data/dev/'
dev_feat, dev_lab = get_data(path, color=rbg)

100% (909 of 909) |#######################| Elapsed Time: 0:00:10 Time: 0:00:10


In [6]:
if K.image_data_format() == 'channels_first':
    input_shape = (n_channels, img_width, img_height)
else:
    input_shape = (img_width, img_height, n_channels)

# layer 1
model = Sequential()
model.add(Conv2D(512, (5, 5), input_shape=input_shape))
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(MaxPooling2D(pool_size=(2, 2)))

# layer 2
model = Sequential()
model.add(Conv2D(512, (5, 5), input_shape=input_shape))
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(MaxPooling2D(pool_size=(2, 2)))

# layer 3
model = Sequential()
model.add(Conv2D(512, (5, 5), input_shape=input_shape))
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(MaxPooling2D(pool_size=(2, 2)))

# layer 4
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(MaxPooling2D(pool_size=(2, 2)))

# layer 5
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(MaxPooling2D(pool_size=(2, 2)))

# layer 6
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

# layer 7
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.8))

# layer 8
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.8))

# layer 9
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.8))

# layer 10
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

In [7]:
model.fit(
    train_feat, 
    train_lab, 
    batch_size=batch_size, 
    epochs=epochs, 
    verbose=1,
    validation_data=(dev_feat, dev_lab),
    shuffle=True)

Train on 9054 samples, validate on 1818 samples
Epoch 1/5
9054/9054 [==============================] - 417s 46ms/step - loss: 7.3491 - acc: 0.4981 - val_loss: 1.7643 - val_acc: 0.5000
Epoch 2/5
9054/9054 [==============================] - 421s 46ms/step - loss: 7.4873 - acc: 0.5046 - val_loss: 1.1804 - val_acc: 0.5000
Epoch 3/5
9054/9054 [==============================] - 413s 46ms/step - loss: 6.3612 - acc: 0.4964 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 4/5
9054/9054 [==============================] - 1356s 150ms/step - loss: 0.7066 - acc: 0.5081 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 5/5
9054/9054 [==============================] - 419s 46ms/step - loss: 0.6937 - acc: 0.4972 - val_loss: 0.6932 - val_acc: 0.5000


In [8]:
model.save('/Users/alejandro/AnacondaProjects/cv/model_from_edges.h5')

In [9]:
from keras.models import load_model
model2 = load_model('/Users/alejandro/AnacondaProjects/cv/model_from_edges.h5')

In [10]:
def pred_from_file(files, model, color=0):
    predictions = []
    with progressbar.ProgressBar(max_value=len(files)) as bar:
        for i in range(len(files)):
            img = cv2.imread(path + files[i], color)
            img = img.reshape((1, img_width, img_height, n_channels))//255
            pred = model.predict(img)
            pred = np.argmax(pred)
            predictions.append(pred)
            bar.update(i)
    return predictions

In [11]:
# In-sample performance
path = '/Users/alejandro/Dropbox (Work)/Hand_Data/train/hand/'
files = os.listdir(path)
pred_hand = pred_from_file(files, model2, color=rbg)

100% (4527 of 4527) |#####################| Elapsed Time: 0:00:57 Time: 0:00:57


In [12]:
path = '/Users/alejandro/Dropbox (Work)/Hand_Data/train/no_hand/'
files = os.listdir(path)
pred_no_hand = pred_from_file(files, model2, color=rbg)

100% (5360 of 5360) |#####################| Elapsed Time: 0:01:07 Time: 0:01:07


In [13]:
pred_no_hand[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [14]:
FN = round(np.sum(pred_hand)/len(files), 4) * 100
FP = 100 - round(np.sum(pred_no_hand)/len(files), 4) * 100
TN = 100 - FN
TP = 100 - FP
Acc = (TP + TN) / (TP + TN + FP + FN)

print('In-sample performance:')
print('False negatives rate:', FN)
print('False positive rate:', FP)
print('Accuracy:', Acc)

In-sample performance:
False negatives rate: 0.0
False positive rate: 100.0
Accuracy: 0.5


In [15]:
# Out of sample performance
path = '/Users/alejandro/Dropbox (Work)/Hand_Data/dev/hand/'
files = os.listdir(path)
pred_hand = pred_from_file(files, model2, color=rbg)

100% (909 of 909) |#######################| Elapsed Time: 0:00:11 Time: 0:00:11


In [16]:
path = '/Users/alejandro/Dropbox (Work)/Hand_Data/dev/no_hand/'
files = os.listdir(path)
pred_no_hand = pred_from_file(files, model2, color=rbg)

100% (1102 of 1102) |#####################| Elapsed Time: 0:00:13 Time: 0:00:13


In [17]:
FN = round(np.sum(pred_hand)/len(files), 4) * 100
FP = 100 - round(np.sum(pred_no_hand)/len(files), 4) * 100
TN = 100 - FN
TP = 100 - FP
Acc = (TP + TN) / (TP + TN + FP + FN)

print('Out-of-sample performance:')
print('False negatives rate:', FN)
print('False positive rate:', FP)
print('Accuracy:', Acc)

Out-of-sample performance:
False negatives rate: 0.0
False positive rate: 100.0
Accuracy: 0.5
